In [42]:
import pandas as pd
import numpy as np
from scipy import optimize
import numpy_financial as npf
import itertools 

#Ponemos la información de entrada

#Datos de los captadores

A=1.9     #Área de un colector solar
N= 6      #Número de captadores
sc= A*N   #Área predimensionada


#Datos del colector
n_1= 0.696 #Eficiencia óptica del colector
n_2= 0.94  #Modificador del ángulo de incidencia
n_3= 0.95 #Factor de corrección del conjunto intercambiador
U= 4.821   #Coeficiente de pérdidas lineales

F_R_t= n_1*n_2*n_3
F_R_U_l= n_3*U


#Producción de CO2
CO2_ratio= 0.41    #CO2equi/kWh

#Costos
sol_ratio= 0.77   #soles/kWh
cos_sc= 320          #soles/m2
cos_V= 0.72          #soles/kg

#Importamos datos mensuales de la temperatura de agua de red, radiación media diaria por unidad de superficie, Factor de corrección sobre superficie inclinada
case_thermal= pd.read_csv("DesignBuilder\Casos demanda\Results demand\case 7 - thermal demand - hourly.csv", encoding= "latin-1", sep=";")
case_thermal=case_thermal.drop([0], axis=0)

#Set the date column as index 
case_thermal["Date/Time"]= pd.to_datetime(case_thermal["Date/Time"])
case_thermal["Zone Heating"]= pd.to_numeric(case_thermal["Zone Heating"])
case_thermal["DHW (Electricity)"]= pd.to_numeric(case_thermal["DHW (Electricity)"])

case_thermal= case_thermal.set_index("Date/Time")

df_monthly_sum= case_thermal[["Zone Heating", "DHW (Electricity)"]].resample("M").sum()

df_monthly_sum["Total_demand_kwh"]= df_monthly_sum["Zone Heating"]+ df_monthly_sum["DHW (Electricity)"]
df_monthly_sum["Total_demand_MJ"] = df_monthly_sum["Total_demand_kwh"]*3.6

t_mensual= pd.read_csv("input_month.csv", sep=";")
t_mensual["Total_demand_MJ"]= list(df_monthly_sum["Total_demand_MJ"])


f_value=[]
aporte_solar_MJ=[]
rendimiento=[]
backup_SOL=[]
CO2_kg=[]
ahorro=[]
number_collector=[]
tir=[]
inversion= []
VAN=[]
payback=[]
opt_cost=pd.DataFrame(columns=["N_colectors","income",'outcome',"cash_flow"])


interest_rate = 0.1  # The default guess value

#Cálculo para cada una de las áreas 

for N in range (1,21):
    #Calculations
    sc= A*N   #Área predimensionada (m2)
    V= A*75   #Volumen predimensionado (l)
    invertion= sc*cos_sc+V*cos_V
    t_mensual["X"]=F_R_U_l*3600*(100-t_mensual["Temp_outdoor"])*t_mensual["Sun_hours"]*t_mensual["Days_month"]*sc/(t_mensual["Total_demand_MJ"]*1000000)
    t_mensual["Y"]= n_1*n_3*n_2*t_mensual["Rad_enegy_MJ"]*t_mensual["Days_month"]*1000000*sc/(t_mensual["Total_demand_MJ"]*1000000)
    t_mensual["f_value"]=1.020*t_mensual["Y"]-0.065*t_mensual["X"]-0.245*t_mensual["Y"].pow(2)+ 0.0018*t_mensual["X"]+0.0215*t_mensual["Y"].pow(3)
    t_mensual["aporte_solar_MJ"]= t_mensual["f_value"]*t_mensual["Total_demand_MJ"]
    t_mensual["rendimiento"]= t_mensual["aporte_solar_MJ"]/(t_mensual["Rad_enegy_MJ"]*t_mensual["Days_month"]*sc)
    t_mensual["backup_cost_SOL"]= (t_mensual["Total_demand_MJ"]-t_mensual["aporte_solar_MJ"])*sol_ratio*0.2777
    t_mensual["CO2_kg"]= (t_mensual["Total_demand_MJ"]-t_mensual["aporte_solar_MJ"])*CO2_ratio
    t_mensual["Ahorro"]=(t_mensual["Total_demand_MJ"]-t_mensual["aporte_solar_MJ"])*sol_ratio*0.277
    
    #Important parameters results
    f_value.append(t_mensual["aporte_solar_MJ"].sum()/t_mensual["Total_demand_MJ"].sum())
    rendimiento.append(t_mensual["rendimiento"].mean())
    aporte_solar_MJ.append(t_mensual.aporte_solar_MJ.sum())
    number_collector.append(N)
    backup_SOL.append(t_mensual["backup_cost_SOL"].sum())
    CO2_kg.append(t_mensual["CO2_kg"].sum())
    inversion.append(invertion)
    

    #Calculations of the cash flows
    
    income=[]
    outcome=[]
    years=[]
    N_col=[]
    
    for a in range (1,6):
        outcome.append(t_mensual["backup_cost_SOL"].sum())
        income.append((t_mensual["Total_demand_MJ"]*0.277*sol_ratio).sum())
        years.append(a)
        N_col.append(N)
 
    balance= pd.DataFrame({'income':income,'outcome':outcome, "years":years, "N_colectors":N_col})
    balance["cash_flow"]= balance["income"]-balance["outcome"]
    
    cash= [invertion*-1]
    cash.append(list(balance["cash_flow"]))
    flat_list = itertools.chain(*cash)
    flat_list=[]
        
    def flatten_list (data):
    
        for element in data:
            if type(element) == list:
                flatten_list(element)
            else:
                flat_list.append(element)
            
    flatten_list(cash)
    
    VAN_i= npf.npv(interest_rate,flat_list)
    VAN.append(VAN_i)
    
    tir_i= npf.irr(flat_list)
    tir.append(tir_i)
    
    
    cumulative_cash_flows = 0
    payback_period= 0
    
    while cumulative_cash_flows < invertion:
        cumulative_cash_flows += flat_list [payback_period]
        payback_period+=1
        
        if payback_period== len(flat_list):
            break
    payback.append(payback_period)
df_technical_results=pd.DataFrame(list(zip(number_collector,rendimiento, f_value,aporte_solar_MJ, backup_SOL, CO2_kg,inversion, VAN, tir, payback)), columns=["number_collector","rendimiento","f_value","aporte_solar_MJ", "backup_SOL", "CO2_kg", "inversion", "VAN", "TIR", "payback"])
df_technical_results.to_csv("CleanData/tech_results_df.csv")
details_of_costs= opt_cost.to_csv("CleanData/detail_costs_df.csv")

    

C:\Users\mayte\AppData\Local\Temp\ipykernel_13664\3925944724.py:39: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  case_thermal["Date/Time"]= pd.to_datetime(case_thermal["Date/Time"])


Leemos los datos del cálculo de demanda térmica del caso seleccionado

In [29]:
import pandas as pd
case_thermal= pd.read_csv("DesignBuilder\Casos demanda\Results demand\case 7 - thermal demand - hourly.csv", encoding= "latin-1", sep=";")
case_thermal=case_thermal.drop([0], axis=0)

In [30]:
case_thermal= pd.read_csv("DesignBuilder\Casos demanda\Results demand\case 7 - thermal demand - hourly.csv", encoding= "latin-1", sep=";")
case_thermal=case_thermal.drop([0], axis=0)

#Set the date column as index 
case_thermal["Date/Time"]= pd.to_datetime(case_thermal["Date/Time"])
case_thermal["Zone Heating"]= pd.to_numeric(case_thermal["Zone Heating"])
case_thermal["DHW (Electricity)"]= pd.to_numeric(case_thermal["DHW (Electricity)"])

case_thermal= case_thermal.set_index("Date/Time")

df_monthly_sum= case_thermal[["Zone Heating", "DHW (Electricity)"]].resample("M").sum()

df_monthly_sum["Total_demand_kwh"]= df_monthly_sum["Zone Heating"]+ df_monthly_sum["DHW (Electricity)"]
df_monthly_sum["Total_demand_MJ"] = df_monthly_sum["Total_demand_kwh"]*3.6

t_mensual= pd.read_csv("input_month.csv", sep=";")
t_mensual["Total_demand_MJ"]= list(df_monthly_sum["Total_demand_MJ"])

C:\Users\mayte\AppData\Local\Temp\ipykernel_13664\93702665.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  case_thermal["Date/Time"]= pd.to_datetime(case_thermal["Date/Time"])


In [34]:
df_monthly_sum["Total_demand_kwh"]= df_monthly_sum["Zone Heating"]+ df_monthly_sum["DHW (Electricity)"]
df_monthly_sum["Total_demand_MJ"] = df_monthly_sum["Total_demand_kwh"]*3.6

In [40]:
t_mensual= pd.read_csv("input_month.csv", sep=";")
t_mensual["Total_demand_MJ"]= list(df_monthly_sum["Total_demand_MJ"])

In [41]:
t_mensual

,Month,Month_numb,Days_month,Temp_red,Rad_enegy_MJ,Temp_outdoor,Sun_hours,Thermal_demand_MJ,Total_demand_MJ
0,Enero,1,31,12.53,7.8,9,8,4830.587455,1322.260790
1,Febrero,2,28,12.58,10.8,10,7,4358.515598,1459.636690
2,Marzo,3,31,12.55,15.2,13,9,4828.552238,1422.218174
3,Abril,4,30,12.35,18.5,16,5,4692.488136,1690.859585
4,Mayo,5,31,11.37,21.9,18,8,4948.630038,2008.591913
5,Junio,6,30,10.35,24.8,24,7,4889.444616,2322.477245
6,Julio,7,31,9.93,26.7,27,8,5095.165659,2268.235595
7,Agosto,8,31,10.96,23.6,27,9,4990.351986,1915.055770
8,Setiembre,9,30,12.09,18.8,24,9,4718.092478,1670.999299
9,Octubre,10,31,13.21,12.9,18,10,4761.390078,1405.452546
